In [1]:
import tensorflow as tf
import torch 

In [2]:
def _independent_straight_through_sampling(rationale_logits):
    """
    Straight through sampling.
    Outputs:
        z -- shape (batch_size, sequence_length, 2)        
    """
    z = tf.nn.softmax(rationale_logits)
    z_hard = tf.cast(tf.equal(z, tf.reduce_max(z, -1, keep_dims=True)),
                     z.dtype)
    z = tf.stop_gradient(z_hard - z) + z

    return z

In [4]:
rationale_logits = [
    [0.9, 0.1],
    [0.8, 0.2],
    [0.7, 0.3]
]
rationale_logits = torch.tensor(rationale_logits)
z = torch.nn.functional.softmax(rationale_logits)
print(z)
print("\n")
reduce_max = torch.max(z, dim = 1, keepdim=False).values.reshape(-1,1)
print(reduce_max)
equal = z.eq(reduce_max)
z_hard = equal.type(torch.FloatTensor)
z_soft = (z_hard - z).detach_() + z
print(z_soft)

tensor([[0.6900, 0.3100],
        [0.6457, 0.3543],
        [0.5987, 0.4013]])


tensor([[0.6900],
        [0.6457],
        [0.5987]])
tensor([[1., 0.],
        [1., 0.],
        [1., 0.]])


<ipython-input-4-2a0d3a285849>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = torch.nn.functional.softmax(rationale_logits)


In [26]:
rationale_logits.shape

torch.Size([3, 2])

In [29]:
weight = torch.tensor([1,0])

In [30]:
rationale_logits*weight

tensor([[0.9000, 0.0000],
        [0.8000, 0.0000],
        [0.7000, 0.0000]])

In [6]:
import numpy as np
X = np.array([1,2,3,4,5,6,7,8,9])
Y = torch.tensor(X)
index_ = [1,4,6]
X[index_], Y[index_]

(array([2, 5, 7]), tensor([2, 5, 7]))

In [8]:
from torch.autograd import Variable


In [11]:
Variable(torch.tensor([1]*10), requires_grad=True)

RuntimeError: Only Tensors of floating point and complex dtype can require gradients

In [14]:
W = torch.nn.Parameter(torch.randn(5))
W.requires_grad = True

In [15]:
W

Parameter containing:
tensor([ 0.0492, -0.2416, -0.4058,  1.7760,  0.9588], requires_grad=True)

In [16]:
1.3219*0.67

0.8856730000000002

In [17]:
torch.max(torch.tensor(1), torch.tensor(0))

tensor(1)

In [18]:
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [19]:
input

tensor([[-1.3380, -1.5622,  0.2470, -2.5529, -0.2602],
        [ 0.5406, -0.7182, -1.1205, -0.8201, -0.7561],
        [ 0.2732,  0.7713, -0.6576,  0.8543, -1.3784]], requires_grad=True)

In [20]:
target

tensor([4, 3, 1])

In [22]:
m = torch.nn.Sigmoid()
loss = torch.nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)

In [23]:
input

tensor([-0.2614, -0.6342, -0.3514], requires_grad=True)

In [24]:
target

tensor([0., 1., 1.])

In [26]:
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [27]:
output

tensor(2.0806, grad_fn=<NllLossBackward>)

In [28]:
input

tensor([[-0.0093, -0.3117, -0.4509, -0.1809, -1.4805],
        [-0.9836, -0.2253,  1.1149, -0.0768, -0.9962],
        [-0.6079, -0.1123, -2.2882, -1.1137, -1.3450]], requires_grad=True)

In [29]:
target

tensor([2, 0, 3])

In [30]:
target

tensor([2, 0, 3])

In [31]:
import torch
import torch.nn as nn
from torch.autograd import Variable

output = Variable(torch.rand(1,10))
target = Variable(torch.LongTensor([1]))

criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print(loss)

tensor(2.4598)


In [32]:
output

tensor([[7.1111e-01, 3.7134e-01, 4.2188e-01, 9.3949e-01, 3.5060e-01, 2.4225e-01,
         4.2464e-02, 7.3067e-01, 9.4832e-01, 6.4373e-05]])

In [33]:
target

tensor([1])

In [35]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = torch.nn.functional.cross_entropy(input, target)

In [36]:
input

tensor([[-1.6440,  0.1403,  0.6018,  1.0484, -0.0409],
        [ 0.0140,  0.0413,  0.0205, -1.1846, -0.0962],
        [ 0.5546,  0.7451,  0.0157, -1.9727,  0.9452]], requires_grad=True)

In [37]:
target

tensor([4, 3, 0])

In [43]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

df = pd.read_csv("/data/trduong/counterfactual_fairness_game_theoric/data/raw/law_data.csv")

# DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None, *, prefetch_factor=2,
#            persistent_workers=False)

In [54]:
train_x, train_y = df[['LSAT', 'UGPA']].values, df['ZFYA'].values

In [55]:
train_x

array([[39. ,  3.1],
       [36. ,  3. ],
       [30. ,  3.1],
       ...,
       [36. ,  4. ],
       [31. ,  2.9],
       [28. ,  3.1]])

In [56]:
train_x, train_y = torch.from_numpy(train_x), torch.from_numpy(train_y)
data_set = TensorDataset(train_x, train_y)
train_batches = DataLoader(data_set, batch_size=1024, shuffle=True)


In [57]:
for x_batch, y_batch in train_batches:
    print(x_batch, y_batch)

tensor([[39.0000,  3.1000],
        [36.0000,  3.0000],
        [30.0000,  3.1000],
        ...,
        [39.0000,  3.4000],
        [36.0000,  3.5000],
        [39.0000,  3.1000]], dtype=torch.float64) tensor([-0.9800,  0.0900, -0.3500,  ...,  0.1900, -0.2400, -0.0300],
       dtype=torch.float64)
tensor([[39.0000,  3.5000],
        [35.0000,  3.3000],
        [36.0000,  3.3000],
        ...,
        [37.0000,  2.1000],
        [33.0000,  2.7000],
        [39.0000,  2.2000]], dtype=torch.float64) tensor([ 1.1100, -0.5200,  0.7200,  ..., -0.0100,  0.5700,  0.4300],
       dtype=torch.float64)
tensor([[34.0000,  2.7000],
        [24.7000,  2.6000],
        [34.0000,  2.9000],
        ...,
        [41.0000,  3.3000],
        [38.0000,  3.6000],
        [34.0000,  3.1000]], dtype=torch.float64) tensor([ 1.1900, -1.5400,  1.6100,  ...,  1.5200,  0.1500, -0.5700],
       dtype=torch.float64)
tensor([[35.0000,  2.9000],
        [39.0000,  3.5000],
        [31.0000,  3.3000],
        ...,
   

In [ ]:
Prediction  [ 0.12912953  0.04191843  0.03074031 ...  0.11929967  0.02832243
 -0.14785084]
Label  [-1.21691224 -0.05936492 -0.53536568 ...  2.13672954  0.16781727
  0.12454447]